# Mongo db built in full text search
* time complexity: O(log(n))
* Uses full-text search capabilities
* Performs stemming (reducing words to their root form)
* Ignores stop words (common words like "the", "a", "an")
* Supports phrase searches and negation

In [ ]:
from mongo import Mongo

from pymongo import MongoClient

class MongoSentenceSearch:
    def __init__(self, db_name='sentence_search', collection_name='sentences'):
        mongo = Mongo()
        self.collection = mongo.db['test']
        
        # Ensure text index is created
        self.collection.create_index([('text', 'text')])
    def add_sentences(self, sentences):
        docs = [{'text': sentence} for sentence in sentences]
        self.collection.insert_many(docs)
    def search(self, query):
        results = self.collection.find(
            {'$text': {'$search': query}},
            {'score': {'$meta': 'textScore'}, 'text': 1}
        ).sort([('score', {'$meta': 'textScore'})])
        return [(doc['text'], doc['score']) for doc in results]

# Example usage
searcher = MongoSentenceSearch()

# Add sentences to MongoDB
sentences = ["I love cats", "I love dogs too", "Cats are cute", "Dogs are loyal"]
searcher.add_sentences(sentences)

# Perform search
query = "love dogs"
results = searcher.search(query)
print(f"Results for '{query}':")
for text, score in results:
    print(f"Score: {score:.2f} - Text: {text}")

'''
# Output:
Score: 1.50 - Text: I love dogs too
Score: 1.50 - Text: I love dogs too
Score: 1.50 - Text: I love dogs too
Score: 0.75 - Text: Dogs are loyal
Score: 0.75 - Text: I love cats
Score: 0.75 - Text: Dogs are loyal
Score: 0.75 - Text: I love cats
Score: 0.75 - Text: Dogs are loyal
Score: 0.75 - Text: I love cats
'''